In [3]:
import numpy as np

# 원본 행렬 R 생성, 분해 행렬 P와 Q 초기화, 잠재 요인 차원 K는 3으로 설정.
R = np.array([[4, np.NaN, np.NaN, 2, np.NaN],
              [np.NaN, 5, np.NaN, 3, 1],
              [np.NaN, np.NaN,3, 4, 4],
              [5, 2, 1, 2, np.NaN]])


num_users, num_items = R.shape
K = 3

# P와 Q 행렬의 크기를 지정하고 정규 분포를 가진 임의의 값으로 입력합니다.
np.random.seed(1)
P = np.random.normal(scale = 1./K, size=(num_users, K))
Q = np.random.normal(scale = 1./K, size=(num_items, K))

In [8]:
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두 개의 분해된 행렬 P와 Q.T의 내적으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P, Q.T)

    # 실제 R 행렬에서 널이 아닌 값의 위치 인덱스 추출해 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind, y_non_zero_ind]
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)

    return rmse

In [10]:
# R > 0 인 행 위치, 열 위치, 값을 non_zeros 리스트에 저장.
non_zeros = [(i, j, R[i, j]) for i in range(num_users) for j in range(num_items) if R[i, j] > 0]

steps = 1000
learning_rate = 0.01
r_lambda = 0.01

# SGD 기법으로 P와 Q 매트릭스를 계속 업데이트

for step in range(steps):
    for i, j, r in non_zeros:
        # 실제 값과 예측 값의 차이인 오류 값 구함
        eij = r - np.dot(P[i, :], Q[j, :].T)
        # Regularization을 반영한 SGD 업데이트 공식 적용
        P[i,:] = P[i,:] + learning_rate*(eij*Q[j,:] - r_lambda*P[i,:])
        Q[j,:] = Q[j,:] + learning_rate*(eij*P[i,:] - r_lambda*Q[j,:])

        rmse = get_rmse(R, P, Q, non_zeros)
        if (step % 50) == 0 :
            print('### iteration step:', step, 'rmse', rmse)



### iteration step: 0 rmse 3.245759961036056
### iteration step: 0 rmse 3.2448480183778514
### iteration step: 0 rmse 3.2382551404813538
### iteration step: 0 rmse 3.2364207044230158
### iteration step: 0 rmse 3.2366077875750956
### iteration step: 0 rmse 3.2353385884132098
### iteration step: 0 rmse 3.23350624157706
### iteration step: 0 rmse 3.2316735169827373
### iteration step: 0 rmse 3.2264716233710518
### iteration step: 0 rmse 3.2246830533090827
### iteration step: 0 rmse 3.2247898624415567
### iteration step: 0 rmse 3.2233919867537426
### iteration step: 50 rmse 0.49318423041111104
### iteration step: 50 rmse 0.4929307859379674
### iteration step: 50 rmse 0.4919654608805921
### iteration step: 50 rmse 0.4917702706133917
### iteration step: 50 rmse 0.49187075299214444
### iteration step: 50 rmse 0.49121116264303544
### iteration step: 50 rmse 0.491304852504746
### iteration step: 50 rmse 0.49082404221805437
### iteration step: 50 rmse 0.4832102816219243
### iteration step: 50 rm

In [11]:
pred_matrix = np.dot(P, Q.T)
print('예측 행렬:\n', np.round(pred_matrix, 3))

예측 행렬:
 [[3.99  0.692 1.369 2.003 1.778]
 [6.679 4.978 0.938 2.98  1.003]
 [7.081 0.437 2.987 3.976 3.986]
 [4.967 2.004 1.007 2.019 1.179]]
